In [49]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [50]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [51]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [52]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [53]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [55]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 40


In [56]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Progreso,MX,2021-12-04 17:19,21.2833,-89.6667,27.84,58,43,4.07
16,16,Butaritari,KI,2021-12-04 17:19,3.0707,172.7902,28.00,77,32,7.43
22,22,Gombong,ID,2021-12-04 17:19,-7.6072,109.5142,28.24,87,100,0.61
30,30,Arraial Do Cabo,BR,2021-12-04 17:19,-22.9661,-42.0278,26.98,54,40,4.63
32,32,Puerto Maldonado,PE,2021-12-04 17:19,-12.6000,-69.1833,29.22,65,40,3.60
41,41,Road Town,VG,2021-12-04 17:16,18.4167,-64.6167,30.02,70,40,4.12
42,42,La Orilla,MX,2021-12-04 17:19,17.9833,-102.2333,28.23,62,57,1.17
43,43,Djenne,ML,2021-12-04 17:19,13.9061,-4.5533,34.23,10,85,2.68
46,46,Zemio,CF,2021-12-04 17:20,5.0314,25.1361,26.81,48,100,1.35
51,51,Esmeraldas,BR,2021-12-04 17:20,-19.7625,-44.3139,28.06,65,81,1.86


In [57]:
preferred_cities_df.count()

City_ID       166
City          166
Country       166
Date          166
Lat           166
Lng           166
Max Temp      166
Humidity      166
Cloudiness    166
Wind Speed    166
dtype: int64

In [58]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Progreso,MX,27.84,21.2833,-89.6667,
16,Butaritari,KI,28.00,3.0707,172.7902,
22,Gombong,ID,28.24,-7.6072,109.5142,
30,Arraial Do Cabo,BR,26.98,-22.9661,-42.0278,
32,Puerto Maldonado,PE,29.22,-12.6000,-69.1833,
41,Road Town,VG,30.02,18.4167,-64.6167,
42,La Orilla,MX,28.23,17.9833,-102.2333,
43,Djenne,ML,34.23,13.9061,-4.5533,
46,Zemio,CF,26.81,5.0314,25.1361,
51,Esmeraldas,BR,28.06,-19.7625,-44.3139,


In [59]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [60]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Progreso,MX,27.84,21.2833,-89.6667,Playa Linda Hotel
16,Butaritari,KI,28.00,3.0707,172.7902,Isles Sunset Lodge
22,Gombong,ID,28.24,-7.6072,109.5142,Grafika Hotel Dan Restoran
30,Arraial Do Cabo,BR,26.98,-22.9661,-42.0278,Pousada Porto Praia
32,Puerto Maldonado,PE,29.22,-12.6000,-69.1833,Wasai Hotel
...,...,...,...,...,...,...
543,Port-Gentil,GA,27.98,-0.7193,8.7815,Fengshui Residence
555,Tual,ID,28.18,-5.6667,132.7500,Grand Vilia Hotel
557,Sabang,ID,27.18,5.8933,95.3214,Hotel Citra
565,Mattru,SL,27.23,7.6244,-11.8332,"Mondkor Town, Bagbo Chiefdom, Bo District"


In [69]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=200, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [73]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [74]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))